In [1]:
from etl.load_dataset import DatasetProcessor, get_tf_eggim_patch_dataset
from custom_models.cnns import base_resnet50
import tensorflow as tf
from keras.metrics import Precision, Recall, AUC, CategoricalAccuracy
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.model_selection import StratifiedKFold

2025-01-24 13:46:44.283566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 13:46:44.317172: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 13:46:44.327243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 13:46:44.352702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-24 13:46:46.195855: W tensorflow/compiler/tf2

In [2]:
target_dir = '../2025-01_EGGIM_Dataset3'
patient_ids = np.load('configs/new_patient_ids.npy', allow_pickle=True)
dp = DatasetProcessor(target_dir)
df = dp.process()
togas_ids_boolean = np.array([x.startswith('PT') for x in df['patient_id'].values])
df_togas = df[togas_ids_boolean].reset_index(drop=True)
df_ipo = df[~togas_ids_boolean].reset_index(drop=True)


In [3]:
batch_size = 32
learning_rate = 1e-4
num_folds = 5
n_classes =3

name = f'../logs/cv_patient_resnet_multi_{num_folds}'

In [4]:
model_weights_directory = '../logs/'
model_fold_dir = sorted(os.listdir(model_weights_directory))

split = dp.patient_wise_split_x_fold(df_togas,df_ipo,patient_ids,n_folds=num_folds,internal_train_size=0.9,target_variable='eggim_square',random_state=42)
eggim_values = []
eggim_per_region = {'vi' : [], 'vii' : [], 'viii' : [], 'ix' : [], 'ii' : []}

for fold, (df_train, df_val, df_test) in enumerate(split):
    patients = df_test['patient_id'].unique()

    n_classes = 3  # Replace with the number of classes you have
    model = base_resnet50(input_shape=(224, 224, 3), n_classes=n_classes)
    # Compile the model with Adam optimizer 13:21
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=[CategoricalAccuracy(name='cat_accuracy'), Precision(name='precision'),
                           Recall(name='recall'),
                           AUC(name='auc')])


    model.load_weights('../logs/' + model_fold_dir[fold] + '/weights.weights.h5')
    
    for patient in patients:
        df_patient = df_togas[df_togas['patient_id'] == patient]
        df_patient = df_patient[~df_patient['landmark'].duplicated(keep='first')]
        landmarks = df_patient['landmark'].to_list()
        tf_df_patient = get_tf_eggim_patch_dataset(df_patient,
                                            num_classes=3,
                                            preprocess_fn=tf.keras.applications.resnet.preprocess_input) 
        tf_df_patient = tf_df_patient.batch(batch_size)

        y_true = np.concatenate([y for (_, y) in tf_df_patient])
        y_pred = model.predict(tf_df_patient)
        y_true_ordinal = np.argmax(y_true, axis=-1) # [0 0 1] -> 2
        y_pred_ordinal = np.argmax(y_pred, axis=-1)

        eggim_true = 0
        eggim_pred = 0

        first_group = 0
        first_group_value_true = 0
        first_group_value_pred = 0

        last_group = 0
        last_group_value_true = 0
        last_group_value_pred = 0

        for i in range(len(landmarks)):
            if landmarks[i][:2] == 'vi' :
                eggim_true += y_true_ordinal[i]
                eggim_pred += y_pred_ordinal[i]
                if landmarks[i][:4] == 'viii':
                    eggim_per_region['viii'].append([y_true_ordinal[i],y_pred_ordinal[i]])
                elif landmarks[i][:3] == 'vii':
                    eggim_per_region['vii'].append([y_true_ordinal[i],y_pred_ordinal[i]])
                else:
                    eggim_per_region['vi'].append([y_true_ordinal[i],y_pred_ordinal[i]])
            elif landmarks[i][:2] == 'ix' or landmarks[i][:2] == 'x.':
                if first_group == 0:
                    first_group = 1
                    first_group_value_true = y_true_ordinal[i]
                    first_group_value_pred = y_pred_ordinal[i]
                else:
                    first_group_value_true = (first_group_value_true + y_true_ordinal[i]) /2
                    first_group_value_pred = (first_group_value_pred + y_pred_ordinal[i]) /2
            else:
                if last_group == 0:
                    last_group = 1
                    last_group_value_true = y_true_ordinal[i]
                    last_group_value_pred = y_pred_ordinal[i]
                else:
                    last_group_value_true = (last_group_value_true + y_true_ordinal[i]) /2 
                    last_group_value_pred = (last_group_value_pred + y_pred_ordinal[i]) /2

        eggim_true += first_group_value_true
        eggim_pred += first_group_value_pred
        eggim_true += last_group_value_true
        eggim_pred += last_group_value_pred

        eggim_per_region['ix'].append([int(first_group_value_true),int(first_group_value_pred)])
        eggim_per_region['ii'].append([int(last_group_value_true),int(last_group_value_pred)])

        eggim_values.append([patient,eggim_true,eggim_pred]) 
    
;

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-01-24 13:46:52.767550: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


2025-01-24 13:46:54.660846: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step


2025-01-24 13:46:55.788245: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step


2025-01-24 13:47:44.819620: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step


''

In [6]:
eggim_values

[['PT004', 1.0, 7.5],
 ['PT005', 3.0, 10.0],
 ['PT006', 2.0, 10.0],
 ['PT011', 5.0, 10.0],
 ['PT013', 1.0, 6.5],
 ['PT015', 0.0, 10.0],
 ['PT016', 0.0, 9.0],
 ['PT020', 2.0, 7.0],
 ['PT023', 1.5, 8.0],
 ['PT024', 7.0, 10.0],
 ['PT030', 0.5, 8.0],
 ['PT032', 4.0, 8.5],
 ['PT037', 1.0, 9.0],
 ['PT021', 8.0, 10.0],
 ['PT025', 3.0, 9.0],
 ['PT026', 2.0, 10.0],
 ['PT028', 5.0, 8.0],
 ['PT029', 7.0, 10.0],
 ['PT031', 0.0, 9.0],
 ['PT036', 2.5, 9.0],
 ['PT041', 1.0, 9.5],
 ['PT046', 1.0, 9.5],
 ['PT047', 0.0, 10.0],
 ['PT048', 1.0, 9.0],
 ['PT049', 1.0, 9.0],
 ['PT059', 4.5, 10.0],
 ['PT009', 9.0, 10.0],
 ['PT022', 8.0, 10.0],
 ['PT034', 3.5, 9.5],
 ['PT039', 2.5, 10.0],
 ['PT040', 6.0, 10.0],
 ['PT043', 5.5, 10.0],
 ['PT051', 0.5, 9.5],
 ['PT061', 1.0, 10.0],
 ['PT062', 4.0, 10.0],
 ['PT063', 1.0, 10.0],
 ['PT064', 1.5, 10.0],
 ['PT065', 0.0, 7.5],
 ['PT066', 1.0, 8.0],
 ['PT035', 8.5, 10.0],
 ['PT038', 3, 10],
 ['PT045', 5.0, 10.0],
 ['PT050', 3.0, 9.5],
 ['PT053', 6.0, 10.0],
 ['PT054', 2.

In [7]:
y_true = []
y_pred = []
for values in eggim_values:
    if values[1] < 5: y_true.append(0)
    else: y_true.append(1)
    if values[2] < 5: y_pred.append(0)
    else: y_pred.append(1)


In [8]:
cm = confusion_matrix(y_true,y_pred)
cm_df = pd.DataFrame(cm, index=[f"True {name}" for name in ['0','1']],
                     columns=[f"Pred {name}" for name in ['0','1']])


In [9]:
cm_df

,Pred 0,Pred 1
True 0,0,49
True 1,0,16


In [10]:
cm

array([[ 0, 49],
       [ 0, 16]])

In [11]:
accuracy = (cm[0][0] + cm[1][1]) / cm.sum()
sensitivity = cm[1][1] / cm[1].sum()
specificity = cm[0][0] / cm[0].sum()

In [35]:
def get_scores(cm):
    acc = []
    sen = np.diag(cm) / np.sum(cm, axis=1)
    spe = []
    for i in range(len(cm)):
        acc.append(cm[i][i] / np.sum(cm[i]))
        true_negatives = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
        false_positives = np.sum(cm[:, i]) - cm[i, i]
        spe.append(true_negatives / (true_negatives + false_positives))

    return np.round(acc,4), np.round(sen,4), np.round(spe,4)

all_regions_true = [row[0] for row in eggim_per_region['ii']]+[row[0] for row in eggim_per_region['ix']]+[row[0] for row in eggim_per_region['vi']]+[row[0] for row in eggim_per_region['vii']]+[row[0] for row in eggim_per_region['viii']]
all_regions_pred = [row[1] for row in eggim_per_region['ii']]+[row[1] for row in eggim_per_region['ix']]+[row[1] for row in eggim_per_region['vi']]+[row[1] for row in eggim_per_region['vii']]+[row[1] for row in eggim_per_region['viii']]

words = ['Acc','Sen','Spe']
f = open('../results_per_region.csv','w')
f.write('Metric,EGGIM 0,EGGIM 1,EGGIM 2,\n')

results = get_scores(confusion_matrix(all_regions_true,all_regions_pred))
i = 0
for result in results:
    f.write(f"Overall {words[i]},{result[0]},{result[1]},{result[2]},\n")
    i += 1

results = get_scores(confusion_matrix([row[0] for row in eggim_per_region['ii']],[row[1] for row in eggim_per_region['ii']]))
i = 0
for result in results:
    f.write(f"ii + xii {words[i]},{result[0]},{result[1]},{result[2]},\n")
    i += 1

results = get_scores(confusion_matrix([row[0] for row in eggim_per_region['ix']],[row[1] for row in eggim_per_region['ix']]))
i = 0
for result in results:
    f.write(f"ix + x {words[i]},{result[0]},{result[1]},{result[2]},\n")
    i += 1

results = get_scores(confusion_matrix([row[0] for row in eggim_per_region['vi']],[row[1] for row in eggim_per_region['vi']]))
i = 0
for result in results:
    f.write(f"vi {words[i]},{result[0]},{result[1]},{result[2]},\n")
    i += 1

results = get_scores(confusion_matrix([row[0] for row in eggim_per_region['vii']],[row[1] for row in eggim_per_region['vii']]))
i = 0
for result in results:
    f.write(f"vii {words[i]},{result[0]},{result[1]},{result[2]},\n")
    i += 1

results = get_scores(confusion_matrix([row[0] for row in eggim_per_region['viii']],[row[1] for row in eggim_per_region['viii']]))
i = 0
for result in results:
    f.write(f"viii {words[i]},{result[0]},{result[1]},{result[2]},\n")
    i += 1


f.close()

(array([0.0667, 0.    , 1.    ]), array([0.0667, 0.    , 1.    ]), array([1.    , 0.7705, 0.2812]))
(array([0.0244, 0.2105, 1.    ]), array([0.0244, 0.2105, 1.    ]), array([1.    , 0.6957, 0.3167]))
(array([0.0645, 0.1667, 1.    ]), array([0.0645, 0.1667, 1.    ]), array([1.    , 0.9362, 0.1633]))
(array([0.0976, 0.    , 1.    ]), array([0.0976, 0.    , 1.    ]), array([0.9167, 0.8727, 0.2549]))
(array([0.  , 0.04, 1.  ]), array([0.  , 0.04, 1.  ]), array([1.    , 0.925 , 0.0769]))


In [12]:
def replace_landmark_name(landmark):
    new_landmark_name = {'ii': 'ii',
                    'ix': 'ix',
                    'vi': 'vi',
                    'vii': 'vii',
                    'viii': 'viii',
                    'x': 'x',
                    'xii': 'xii'}
    landmark_number = landmark.split('.')[0]
    return new_landmark_name[landmark_number]

In [13]:
df_patients = df_togas[df_togas['patient_id'].isin(patient_ids)].drop(['bbox','eggim_global','image_directory'],axis=1)
df_patients = df_patients.drop_duplicates(subset=['patient_id','landmark'])
df_patients['landmark'] = df_patients['landmark'].apply(replace_landmark_name)

In [14]:
filtered_df = df_patients[df_patients['landmark'].isin(['ix','x'])]
filtered_df = filtered_df.groupby(['patient_id'], as_index=False).agg({'eggim_square' : 'mean', 'landmark' : lambda x : 'ix'})

df_patients = df_patients[~df_patients['landmark'].isin(['ix','x'])]

df_patients = pd.concat([df_patients, filtered_df])

In [15]:
filtered_df = df_patients[df_patients['landmark'].isin(['ii','xii'])]
filtered_df = filtered_df.groupby(['patient_id'], as_index=False).agg({'eggim_square' : 'mean', 'landmark' : lambda x : 'ii'})

df_patients = df_patients[~df_patients['landmark'].isin(['ii','xii'])]

df_patients = pd.concat([df_patients, filtered_df])

In [16]:
df_per_patient = df_patients.groupby(['patient_id'], as_index=False).sum()
df_per_patient['eggim_square'] = df_per_patient['eggim_square'].astype(int)


In [17]:
def get_folds(eggim,patients):
    skf = StratifiedKFold(n_splits=5,shuffle=False)
    folds = []
    for _, (_, test_index) in enumerate(skf.split(patients,eggim)):
        p_list = [patients[index] for index in test_index ]
        folds.append(p_list)

    return folds

folds = get_folds(df_per_patient['eggim_square'].to_list(),df_per_patient['patient_id'].to_list())

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
